# Workbook to drive DC motors of a Robot
Two DC motors will be connected to the PYNQ via a TI L293 half bridge driver and be controlled via this workbook.

Here we load the base overlay and import the necessary libraries

In [8]:
from pynq import Overlay
from pynq.iop import Arduino_IO
from pynq.iop import ARDUINO
from IPython import display
import time
from ipywidgets import *

Overlay("base.bit").download()

Here we set up the motors that will drive the robot via the PYNQ's digital IO pins 

In [9]:
#Motor 1
m1_enable = Arduino_IO(ARDUINO, 7, 'out')
m1_left = Arduino_IO(ARDUINO, 6, 'out')
m1_right = Arduino_IO(ARDUINO, 5, 'out')

#Motor 2
m2_enable = Arduino_IO(ARDUINO, 8, 'out')
m2_left = Arduino_IO(ARDUINO, 9, 'out')
m2_right = Arduino_IO(ARDUINO, 10, 'out')


#Ultrasonic sensor 1
TRIG1 = Arduino_IO(ARDUINO, 3, 'out')
#ECHO1 = Arduino_IO(ARDUINO, 4, 'in')

In [10]:
fwd_button = widgets.Button(description="Forward")
fwd_button.width = "120px"
fwd_button.background_color = "#FFFF00"
fwd_button.color = "#ffffff"

back_button = widgets.Button(description="Backward")
back_button.width = "120px"
back_button.background_color = "#FFFF00"
back_button.color = "#ffffff"

left_button = widgets.Button(description="Left")
left_button.width = "120px"
left_button.background_color = "#FFFF00"
left_button.color = "#ffffff"

right_button = widgets.Button(description="Right")
right_button.width = "120px"
right_button.background_color = "#FFFF00"
right_button.color = "#ffffff"

stop_button = widgets.Button(description="Stop")
stop_button.width = "120px"
stop_button.background_color = "#FFFF00"
stop_button.color = "#ffffff"

dummy = widgets.Button(description="")
dummy.width = "120px"
dummy.background_color = "#FFFF00"
dummy.color = "#ffffff"

Method that returns the distance from the ultrasonic sensor

In [11]:
def getDistance():
    #global pulse_start, pulse_end, pulse_duration
    pulse_start = 0
    pulse_end = 0
    #pulse_duration = 0

    distance = 0
    TRIG1.write(0)
    time.sleep(1)
    
    
    TRIG1.write(1)
    time.sleep(0.00001)
    TRIG1.write(0)
    
    while ECHO1.read() == 0:
        pulse_start = time.time()

    while ECHO1.read() == 1: 
        pulse_end = time.time()
    
    try:
        pulse_duration = pulse_end - pulse_start
        distance = pulse_duration * 17150

        distance = round(distance, 2)    
    
        return distance
        
    except UnboundLocalError:
        print("Error", end = '\r')
        pass
    
    return -1

Here are the functions that the buttons will execute when pressed. Each function sends a signal to the H-bridge and drives the motors in divverent ways (forward, backwards, left, right, stop).
Commented code is for ultrasonic sensor

In [12]:
def motor_fwd(b):
    #Check path in front
    distanceAhead = 11
    #distanceAhead = getDistance()
    print("Distance Ahead", distanceAhead, "cm")

    if distanceAhead < 0 or distanceAhead > 500:
        print("Sensor Malfunction - Sensor Reading:", distanceAhead, "cm")
    elif distanceAhead < 10:
        print("Path obstructed, object", distanceAhead, "cm away")
    else:
        print("Path clear, going forward")
        m1_enable.write(1) #enable
        m1_left.write(0)
        m1_right.write(1)

        m2_enable.write(1) #enable
        m2_left.write(0)
        m2_right.write(1)

def motor_back(b):    
    m1_enable.write(1) #enable
    m1_left.write(1)
    m1_right.write(0)

    m2_enable.write(1) #enable
    m2_left.write(1)
    m2_right.write(0)
    print("Reversing")

def motor_left(b):
    m1_enable.write(1) #enable
    m1_left.write(0)
    m1_right.write(1)

    m2_enable.write(1) #enable
    m2_left.write(1)
    m2_right.write(0)
    print("Going Left")
    
def motor_right(b):
    m1_enable.write(1) #enable
    m1_left.write(1)
    m1_right.write(0)

    m2_enable.write(1) #enable
    m2_left.write(0)
    m2_right.write(1)
    print("Going Right")
    
def motor_stop(b):
    m1_enable.write(0) #enable
    m2_enable.write(0) #enable
    print("Stopping")

Map buttons to functions - When a click event occurs, the desired function is executed.

In [13]:
fwd_button.on_click(motor_fwd)
back_button.on_click(motor_back)
left_button.on_click(motor_left)
right_button.on_click(motor_right)
stop_button.on_click(motor_stop)

Display buttons

In [14]:
HBox([VBox([dummy, 
            left_button]), 
      
      VBox([fwd_button, 
            stop_button, 
            back_button]), 
      
      VBox([dummy, 
            right_button])], 
             background_color='#EEE')